In [1]:
import os, sys
import numpy as np

from astropy.io import fits
from astropy.table import Table, join
import pandas as pd

#import desispec
# library location will change ..
#sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
#from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [2]:
VI_dir = os.environ['HOME']+'/Dropbox/DESI/DESI-VI/Mini-SV2/BGS_70502/'

In [3]:
#read in all VI files (previously merged)
df = pd.read_csv(VI_dir + 'vi_merged_BGS_tile70502.csv') #BGS tile 70502
vi = Table.from_pandas(df)

In [4]:
vi.keys()

['TargetID',
 'ExpID',
 'Spec version',
 'Redrock version',
 'Redrock spectype',
 'Redrock z',
 'VI scanner',
 'VI class',
 'VI issue',
 'VI z',
 'VI spectype',
 'VI comment']

In [21]:
#to dataframe
vi_df = vi.to_pandas()

In [22]:
#make new column with best redshift estimate for each VI - take VI redshift if available, else take Redrock redshift. 
#This is not strictly necessary but makes manipulation easier.
vi_df['best redshift'] = vi_df['VI z']
vi_df.loc[vi_df['best redshift']=='--', 'best_redshift'] = vi_df.loc[vi_df['best redshift']=='--', 'Redrock z']
#vi_df['best redshift'] = vi_df['best redshift'].astype(float)

In [23]:
vi_df

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best_redshift
0,35185736609174572,-1,0,0,GALAXY,0.278,acedge,4,--,0.2783,GALAXY,clear lines,0.2783,NaN
1,35185736613364742,-1,0,0,GALAXY,0.441,acedge,4,--,0.4413,GALAXY,LRG,0.4413,NaN
2,35185736613368369,-1,0,0,GALAXY,0.263,acedge,4,--,0.2629,GALAXY,LRG,0.2629,NaN
3,35185736613365274,-1,0,0,GALAXY,0.743,acedge,0,S,--,--,no signal,--,0.743
4,35185736613365164,-1,0,0,GALAXY,0.264,acedge,4,--,0.2635,GALAXY,LRG,0.2635,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,35185730577761118,-1,0,0,GALAXY,0.125,acedge,4,--,0.1253,GALAXY,LRG with weak lines,0.1253,NaN
1228,35185730577760441,-1,0,0,GALAXY,0.254,acedge,4,--,0.2542,GALAXY,LRG,0.2542,NaN
1229,35185730573570209,-1,0,0,GALAXY,0.120,acedge,4,--,0.1201,GALAXY,clear lines,0.1201,NaN
1230,35185730577760335,-1,0,0,GALAXY,0.265,acedge,4,--,0.2651,GALAXY,LRG,0.2651,NaN


In [7]:
#add new columns, holding the mean of the flags and the maximum difference in classification
vi_df['vi_combined_flag'] = vi_df.groupby('TargetID')['VI class'].transform('mean')
vi_df['vi_diff'] = vi_df.groupby('TargetID')['VI class'].transform(lambda x: ( x.max()-x.min()) )

In [8]:
#add new column, with the mean redshift from all values of 'best redshift'
vi_df['vi_combined_z'] = vi_df.groupby('TargetID')['best redshift'].transform('mean')
vi_df['dz'] = vi_df.groupby('TargetID')['best redshift'].transform(lambda x: ( (x.max() - x.min()) / (1+x.mean()) ))

In [9]:
#make groups of visual inspections, grouped by unique objects
vi_gp = vi_df.groupby(['TargetID'])
print('There are ' + str(len(vi_df)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')

There are 1232 visual inspections of a total of 982 unique objects


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033 (problematic VIs)

In [10]:
vi_insp = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].mean()) > 0.0033 ) )
                       & (len(x) >= 2)) #x is a group by TargetID

Get the target IDs of the problematic objects and display in table form (later to add prospect window)

In [11]:
unique_targets = np.unique(vi_insp['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185712449980146 35185712454174940 35185712454175202 35185712454175731
 35185712454176111 35185712454176649 35185712458371294 35185718485585807
 35185718489779354 35185718489780521 35185766816546922 35185766820745119]
Total number of conflicts to resolve:  12


In [12]:
for i in range(len(unique_targets)): 
    display(vi_insp[vi_insp.TargetID==unique_targets[i]])

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
75,35185712449980146,-1,0,0,GALAXY,0.07,KAD,4,--,--,--,--,0.0700,3.0,2,0.070133,0.000374
125,35185712449980146,-1,0,0,GALAXY,0.07,acedge,3,--,0.0704,GALAXY,weak lines but consistent with z=0.07,0.0704,3.0,2,0.070133,0.000374
175,35185712449980146,-1,0,0,GALAXY,0.07,rita,2,--,--,--,blue jump; possible Ha; no other features,0.0700,3.0,2,0.070133,0.000374


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
82,35185712454174940,-1,0,0,QSO,1.496,KAD,1,RS,0.1475,--,"Unusual spectrum"","" no strong features (possib...",0.1475,0.666667,1,1.0465,0.65893
132,35185712454174940,-1,0,0,QSO,1.496,acedge,0,S,--,--,no signal,1.4960,0.666667,1,1.0465,0.65893
182,35185712454174940,-1,0,0,QSO,1.496,rita,1,--,--,--,Mayne noise?,1.4960,0.666667,1,1.0465,0.65893


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
68,35185712454175202,-1,0,0,GALAXY,0.975,KAD,0,R,0.4386,--,"Featureless"","" no continuum"","" possible Ha",0.4386,0.0,0,0.7962,0.29863
118,35185712454175202,-1,0,0,GALAXY,0.975,acedge,0,S,--,--,no signal,0.9750,0.0,0,0.7962,0.29863
168,35185712454175202,-1,0,0,GALAXY,0.975,rita,0,--,--,--,noise only?,0.9750,0.0,0,0.7962,0.29863


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
84,35185712454175731,-1,0,0,GALAXY,0.145,KAD,4,--,--,--,--,0.1450,4.0,0,0.146267,0.003315
134,35185712454175731,-1,0,0,GALAXY,0.145,acedge,4,--,0.1488,GALAXY,clear lines,0.1488,4.0,0,0.146267,0.003315
184,35185712454175731,-1,0,0,GALAXY,0.145,rita,4,--,--,--,--,0.1450,4.0,0,0.146267,0.003315


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
56,35185712454176111,-1,0,0,GALAXY,0.431,KAD,4,--,--,--,--,0.4310,2.333333,3,0.431033,0.00007
106,35185712454176111,-1,0,0,GALAXY,0.431,acedge,2,--,0.4311,GALAXY,continuum mismatch at join of blue and red arms,0.4311,2.333333,3,0.431033,0.00007
156,35185712454176111,-1,0,0,GALAXY,0.431,rita,1,--,--,--,"blue jump; featureless"","" noisy",0.4310,2.333333,3,0.431033,0.00007


...

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
748,35185718485585807,-1,0,0,GALAXY,0.329,acedge,4,--,0.3287,GALAXY,LRG,0.3287,3.0,2,0.32885,0.000226
798,35185718485585807,-1,0,0,GALAXY,0.329,rita,2,--,--,--,"break in the right place"","" possible H&K",0.3290,3.0,2,0.32885,0.000226


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
751,35185718489779354,-1,0,0,GALAXY,0.276,acedge,4,--,0.2764,GALAXY,weak but clear lines,0.2764,3.0,2,0.2762,0.000313
801,35185718489779354,-1,0,0,GALAXY,0.276,rita,2,--,--,--,no strong features,0.2760,3.0,2,0.2762,0.000313


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
891,35185718489780521,-1,0,0,GALAXY,0.118,acedge,4,--,0.1182,GALAXY,LRG with some Hdelta?,0.1182,3.0,2,0.1181,0.000179
941,35185718489780521,-1,0,0,GALAXY,0.118,rita,2,--,--,--,"weak features"","" continuum about right",0.1180,3.0,2,0.1181,0.000179


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
478,35185766816546922,-1,0,0,GALAXY,1.401,KAD,2,R,0.4643,--,--,0.4643,1.0,2,0.93265,0.484671
528,35185766816546922,-1,0,0,GALAXY,1.401,rita,0,--,--,--,--,1.4010,1.0,2,0.93265,0.484671


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
462,35185766820745119,-1,0,0,GALAXY,0.536,KAD,4,R,0.1716,--,--,0.1716,3.0,2,0.3538,0.269168
512,35185766820745119,-1,0,0,GALAXY,0.536,rita,2,--,--,--,single secure line,0.5360,3.0,2,0.3538,0.269168


## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

Suggestion: we edit either 'dz' or 'vi diff' or 'vi_combined_flag' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [13]:
#first, keep a safe copy of the original dataframe
vi_safe = vi_df.copy()

In [14]:
#for example, after inspecting the first problematic VI, I manually edit the fields that are yielding a conflict

#first, remind myself of the problem:
display(vi_df[vi_df.TargetID==unique_targets[0]])

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
75,35185712449980146,-1,0,0,GALAXY,0.07,KAD,4,--,--,--,--,0.0700,3.0,2,0.070133,0.000374
125,35185712449980146,-1,0,0,GALAXY,0.07,acedge,3,--,0.0704,GALAXY,weak lines but consistent with z=0.07,0.0704,3.0,2,0.070133,0.000374
175,35185712449980146,-1,0,0,GALAXY,0.07,rita,2,--,--,--,blue jump; possible Ha; no other features,0.0700,3.0,2,0.070133,0.000374


In [15]:
#now I change either 'best redshift' or 'vi ' or 'vi_combined_flag' to resolve conflict
#in this case there are two conflicts: dz and vi_diff. I'm going to keep Alastair's VI redshift, and keep the combined flag, and remove the problematic vi_diff by setting VI class to VI combined
vi_df.loc[vi_df.TargetID==unique_targets[0], 'best redshift'] = 0.0704
vi_df.loc[vi_df.TargetID==unique_targets[0], 'VI class'] = 3.0


#look at the values again to make sure all is well
display(vi_df[vi_df.TargetID==unique_targets[0]])


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
75,35185712449980146,-1,0,0,GALAXY,0.07,KAD,3.0,--,--,--,--,0.0704,3.0,2,0.070133,0.000374
125,35185712449980146,-1,0,0,GALAXY,0.07,acedge,3.0,--,0.0704,GALAXY,weak lines but consistent with z=0.07,0.0704,3.0,2,0.070133,0.000374
175,35185712449980146,-1,0,0,GALAXY,0.07,rita,3.0,--,--,--,blue jump; possible Ha; no other features,0.0704,3.0,2,0.070133,0.000374


In [16]:
#next one!
display(vi_df[vi_df.TargetID==unique_targets[1]])

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
82,35185712454174940,-1,0,0,QSO,1.496,KAD,1.0,RS,0.1475,--,"Unusual spectrum"","" no strong features (possib...",0.1475,0.666667,1,1.0465,0.65893
132,35185712454174940,-1,0,0,QSO,1.496,acedge,0.0,S,--,--,no signal,1.4960,0.666667,1,1.0465,0.65893
182,35185712454174940,-1,0,0,QSO,1.496,rita,1.0,--,--,--,Mayne noise?,1.4960,0.666667,1,1.0465,0.65893


In [17]:
#now I change either 'best redshift' or 'vi ' or 'vi_combined_flag' to resolve conflict
#in this case I can't make a call without looking at the actual spectra, but let's assume I agree with Kelly! The only conflict is dz, so I only need to change 'best redshift'

vi_df.loc[vi_df.TargetID==unique_targets[1], 'best redshift'] = 0.1475

#look at the values again to make sure all is well
display(vi_df[vi_df.TargetID==unique_targets[1]])

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,vi_combined_flag,vi_diff,vi_combined_z,dz
82,35185712454174940,-1,0,0,QSO,1.496,KAD,1.0,RS,0.1475,--,"Unusual spectrum"","" no strong features (possib...",0.1475,0.666667,1,1.0465,0.65893
132,35185712454174940,-1,0,0,QSO,1.496,acedge,0.0,S,--,--,no signal,0.1475,0.666667,1,1.0465,0.65893
182,35185712454174940,-1,0,0,QSO,1.496,rita,1.0,--,--,--,Mayne noise?,0.1475,0.666667,1,1.0465,0.65893


### and so on...

We should now recompute the conflicts, and not find any (except I didn't resolve everything! but the number of conflicts should now be 10, not 12)

In [18]:
vi_gp = vi_df.groupby(['TargetID'])
vi_insp = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].mean()) > 0.0033 ) )
                       & (len(x) >= 2)) #x is a group by TargetID

In [19]:
unique_targets = np.unique(vi_insp['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185712454175202 35185712454175731 35185712454176111 35185712454176649
 35185712458371294 35185718485585807 35185718489779354 35185718489780521
 35185766816546922 35185766820745119]
Total number of conflicts to resolve:  10


In [20]:
#group by TargetID and make a csv file with the 1st of each group.
vi_gp.first().to_csv(VI_dir+'truth_table_test.txt')